energy consumption vs unemployment: https://www.kaggle.com/jayrav13/unemployment-by-county-us
energy consumption vs storms / weather

In [1]:
import numpy as np
import pandas as pd


In [2]:
""" 
AEP = pd.read_csv("AEP_hourly.csv", memory_map=True)
COMED = pd.read_csv("COMED_hourly.csv", memory_map=True)
DAYTON = pd.read_csv("DAYTON_hourly.csv", memory_map=True)
DEOK = pd.read_csv("DEOK_hourly.csv", memory_map=True)
DOM = pd.read_csv("DOM_hourly.csv", memory_map=True)
DUQ = pd.read_csv("DUQ_hourly.csv", memory_map=True)
EKPC = pd.read_csv("EKPC_hourly.csv", memory_map=True)
FE = pd.read_csv("FE_hourly.csv", memory_map=True)
NI = pd.read_csv("NI_hourly.csv", memory_map=True)
PJM_est = pd.read_csv("pjm_hourly_est.csv", memory_map=True)
PJM_Load = pd.read_csv("PJM_Load_hourly.csv", memory_map=True)
PJME = pd.read_csv("PJME_hourly.csv", memory_map=True)
PJMW = pd.read_csv("PJMW_hourly.csv", memory_map=True)
"""

energy_gdp_by_state_10_14 = pd.read_csv("Census_and_economic_2010_2014.csv", memory_map=True) 
unemployment_90_16 = pd.read_csv("unemployment_by_county.csv", memory_map=True)
gdp_per_cap_92_16 = pd.read_csv("gdp_per_capita.csv", memory_map=True)
gdp_per_cap_92_16.drop('Unnamed: 0', axis=1, inplace=True) 

In [3]:
# Get rid of all years not in our census dataset
unemp_10_14 = unemployment_90_16[(unemployment_90_16['Year']>=2010) & (unemployment_90_16['Year']<=2014)]

# Set index to index column
unemp_10_14.set_index(unemp_10_14.index)

# Remove columns that won't be used
unemp_10_14_short = unemp_10_14.drop(['Month', 'County'], axis=1)

# Set index to start at 0 again
unemp_10_14_short.set_index(np.arange(0,len(unemp_10_14_short), 1), inplace=True)

# Calculate the average unemployment rate for each state, by year. This removes all excess rows to keep just year, state and rate
unemp_10_14_avg = pd.Series.to_frame(unemp_10_14_short.groupby(['Year','State'])['Rate'].mean())
unemp_10_14_avg.reset_index(inplace=True)
unemp_10_14_avg.head()

# Shorten even farther to have 1 row per state. Unemployment rate will then have 1 column for each year. 
# This format is to match the census file

unemp_avg_by_state = unemp_10_14_avg.pivot(index='State', columns='Year', values='Rate')
unemp_avg_by_state.reset_index(inplace=True)
unemp_avg_by_state.columns = ['State', 'Unemp_rate2010', 'Unemp_rate2011', 'Unemp_rate2012', 'Unemp_rate2013', 'Unemp_rate2014']


In [4]:
#Remove excess years not in the main table
gdp_per_cap_10_14 = gdp_per_cap_92_16[(gdp_per_cap_92_16['YEAR'] >= 2010)& (gdp_per_cap_92_16['YEAR'] <= 2014)]
gdp_per_cap_10_14.set_index(np.arange(0,len(gdp_per_cap_10_14),1), inplace=True)

#Pivot table to have 1 row per state like in the main table
gdp_by_state = gdp_per_cap_10_14.pivot(index='STATE', columns='YEAR', values='gdp_per_capita')
gdp_by_state.reset_index(inplace=True)
gdp_by_state.columns = ['State', 'GDP_per_cap2010', 'GDP_per_cap2011', 'GDP_per_cap2012', 'GDP_per_cap2013', 'GDP_per_cap2014']

The main table for this project contains 52 "states", 50 are actual states, and it includes 2 others:

* District of Columbia
* United States

By look at length and unique values, it was found that the unemployment table is missing 5 values from the main table:

* Alaska 
* Florida 
* Georgia 
* District of Columbia
* United States

By looking at the unique values it was found that the GDP table has 8 extra values from the main table:

* Far West
* Great Lakes
* Mideast
* New England
* Plains
* Rocky Mountain
* Southeast
* Southwest

Next step will be to clean up the main census file that contains 192 columns and 50 rows. Any missing data will be found, and correlations between different columns will begin to be explored.

* Can the table to transformed (pivot_table) so that each year is on its own row?
* If this is the way that will be used the pivot transformations of the 2 other tables will be removed
* The 3 tables then need to be merged and explored


In [6]:
first_2 = pd.merge(energy_gdp_by_state_10_14, gdp_by_state, left_on='State', right_on='State', how='left')
df = pd.merge(first_2, unemp_avg_by_state, left_on='State', right_on='State', how='left')
df.head()

,StateCodes,State,Region,Division,Coast,Great Lakes,TotalC2010,TotalC2011,TotalC2012,TotalC2013,...,GDP_per_cap2010,GDP_per_cap2011,GDP_per_cap2012,GDP_per_cap2013,GDP_per_cap2014,Unemp_rate2010,Unemp_rate2011,Unemp_rate2012,Unemp_rate2013,Unemp_rate2014
0,AL,Alabama,3.0,6.0,1.0,0.0,1931522,1905207,1879716,1919365,...,35913,36201,36425,36660,36455,12.360697,11.211070,9.260572,8.507587,7.917164
1,AK,Alaska,4.0,9.0,1.0,0.0,653221,653637,649341,621107,...,69564,70573,73478,69700,67188,NaN,NaN,NaN,NaN,NaN
2,AZ,Arizona,4.0,8.0,0.0,0.0,1383531,1424944,1395839,1414383,...,37935,38257,38559,38303,38450,13.060000,12.219444,11.218333,10.721667,9.470556
3,AR,Arkansas,3.0,7.0,0.0,0.0,1120632,1122544,1067642,1096438,...,34671,35146,34965,35865,36274,9.101111,9.302556,8.532222,8.305000,6.968000
4,CA,California,4.0,9.0,1.0,0.0,7760629,7777115,7564063,7665241,...,51871,52099,52974,53855,55598,13.893276,13.325705,11.637931,10.383621,8.820259


In [20]:
# Separate out columns of our dataframe by year so we can have each row represent a year
cols_2010 = [col for col in df.columns if '2010' in col]
cols_2011 = [col for col in df.columns if '2011' in col]
cols_2012 = [col for col in df.columns if '2012' in col]
cols_2013 = [col for col in df.columns if '2013' in col]
cols_2014 = [col for col in df.columns if '2014' in col]
cols_diff_yrs = [col for col in df.columns if '-' in col]
other_cols = list(set(df.columns) - set(cols_2010) - set(cols_2011) - set(cols_2012) - set(cols_2013) - set(cols_2014) - set(cols_diff_yrs))

['Region', 'Great Lakes', 'Division', 'State', 'StateCodes', 'Coast']


In [21]:
# Get columns of dataframe from previous lists of column names
cols_other_2010 = list(other_cols + cols_2010)
cols_other_2011 = list(other_cols + cols_2011)
cols_other_2012 = list(other_cols + cols_2012)
cols_other_2013 = list(other_cols + cols_2013)
cols_other_2014 = list(other_cols + cols_2014)
df_2010 = df[cols_other_2010]
df_2011 = df[cols_other_2011]
df_2012 = df[cols_other_2012]
df_2013 = df[cols_other_2013]
df_2014 = df[cols_other_2014]
df_2010.head()

,Region,Great Lakes,Division,State,StateCodes,Coast,TotalC2010,TotalP2010,TotalE2010,TotalPrice2010,...,LPGPrice2010,GDP2010Q1,GDP2010Q2,GDP2010Q3,GDP2010Q4,GDP2010,CENSUS2010POP,POPESTIMATE2010,GDP_per_cap2010,Unemp_rate2010
0,3.0,0.0,6.0,Alabama,AL,1.0,1931522,1419613,21630.5,17.82,...,24.20,172033,175198,178520,179132,176220.75,4779736,4785822,35913,12.360697
1,4.0,0.0,9.0,Alaska,AK,1.0,653221,1738207,6473.9,20.13,...,26.79,52307,53673,54846,56054,54220.00,710231,713856,69564,NaN
2,4.0,0.0,8.0,Arizona,AZ,0.0,1383531,580948,19372.7,22.25,...,28.05,242824,246487,248985,251036,247333.00,2915918,2922297,37935,13.060000
3,3.0,0.0,7.0,Arkansas,AR,0.0,1120632,1247709,12269.4,16.90,...,22.40,101063,104880,106687,107085,104928.75,6392017,6411999,34671,9.101111
4,4.0,0.0,9.0,California,CA,1.0,7760629,2532205,117090.0,20.97,...,27.58,1920145,1943428,1963650,2016517,1960935.00,37253956,37336011,51871,13.893276


In [25]:
# Add Year column to each and drop year from individual column names
df_2010['Year'] = 2010
df_2011['Year'] = 2011
df_2012['Year'] = 2012
df_2013['Year'] = 2013
df_2014['Year'] = 2014

df_2010.columns = [col.replace('2010', '') for col in df_2010.columns]
df_2011.columns = [col.replace('2011', '') for col in df_2011.columns]
df_2012.columns = [col.replace('2012', '') for col in df_2012.columns]
df_2013.columns = [col.replace('2013', '') for col in df_2013.columns]
df_2014.columns = [col.replace('2014', '') for col in df_2014.columns]


C:\Users\sarah\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\sarah\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sarah\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\sarah\Anaconda3\lib\

In [27]:
df_full = pd.concat([df_2010, df_2011, df_2012, df_2013, df_2014], ignore_index=True, sort=True)
df_full.head()

,BiomassC,CENSUSPOP,CoalC,CoalE,CoalP,CoalPrice,Coast,Division,ElecC,ElecE,...,RNETMIG,Region,State,StateCodes,TotalC,TotalE,TotalP,TotalPrice,Unemp_rate,Year
0,169088,4779736.0,718684,2135.6,493094,2.97,1.0,6.0,310023,7833.0,...,NaN,3.0,Alabama,AL,1931522,21630.5,1419613,17.82,12.360697,2010
1,4178,710231.0,14548,49.9,33556,3.43,1.0,9.0,21315,912.4,...,NaN,4.0,Alaska,AK,653221,6473.9,1738207,20.13,NaN,2010
2,29289,2915918.0,457909,829.1,167930,1.81,0.0,8.0,248506,7058.8,...,NaN,4.0,Arizona,AZ,1383531,19372.7,580948,22.25,13.060000,2010
3,94865,6392017.0,293689,510.6,718,1.74,0.0,7.0,164439,3393.3,...,NaN,3.0,Arkansas,AR,1120632,12269.4,1247709,16.90,9.101111,2010
4,280124,37253956.0,54972,161.4,0,2.94,1.0,9.0,882107,33381.8,...,NaN,4.0,California,CA,7760629,117090.0,2532205,20.97,13.893276,2010


In [29]:
df_full.describe()

,BiomassC,CENSUSPOP,CoalC,CoalE,CoalP,CoalPrice,Coast,Division,ElecC,ElecE,...,RINTERNATIONALMIG,RNATURALINC,RNETMIG,Region,TotalC,TotalE,TotalP,TotalPrice,Unemp_rate,Year
count,2.650000e+02,5.300000e+01,2.650000e+02,265.000000,2.650000e+02,265.000000,260.000000,260.000000,2.650000e+02,265.000000,...,212.000000,212.000000,212.000000,260.000000,2.650000e+02,2.650000e+02,2.650000e+02,265.000000,240.000000,265.000000
mean,1.649635e+05,1.165081e+07,7.087814e+05,1825.093208,7.890682e+05,40.545811,39.134615,43.711538,4.812949e+05,14050.987925,...,40.385847,42.126451,40.774196,41.307692,3.654578e+06,5.086351e+04,2.931650e+06,59.373434,49.587771,2012.000000
std,5.888774e+05,4.213690e+07,2.545121e+06,6410.787945,3.008260e+06,273.910570,276.789721,276.158377,1.721385e+06,50205.578218,...,274.134662,273.898392,274.111195,276.486333,1.310484e+07,1.825417e+05,1.100403e+07,271.318179,286.854255,1.416889
min,8.190000e+02,2.010000e+03,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,1.000000,2.010000e+03,740.800000,...,0.557539,-0.776457,-10.884734,1.000000,2.010000e+03,2.010000e+03,9.500000e+01,13.790000,3.094713,2010.000000
25%,2.649200e+04,1.567582e+06,3.557400e+04,129.500000,0.000000e+00,2.000000,0.000000,3.000000,7.778600e+04,2012.000000,...,1.376639,2.885390,0.044996,2.000000,6.326550e+05,7.748700e+03,2.366790e+05,19.540000,6.173318,2011.000000
50%,6.520800e+04,4.339367e+06,2.936890e+05,666.200000,7.180000e+02,2.600000,0.000000,5.000000,1.887290e+05,5258.200000,...,1.986608,4.138155,1.768994,3.000000,1.428433e+06,1.967070e+04,6.467380e+05,21.410000,7.860175,2012.000000
75%,1.322400e+05,6.724540e+06,5.015920e+05,1677.300000,4.109970e+05,3.410000,1.000000,8.000000,3.437020e+05,9379.000000,...,3.201814,5.503458,5.274056,4.000000,2.430205e+06,3.415190e+04,1.547368e+06,24.000000,9.293166,2013.000000
max,4.611638e+06,3.087455e+08,2.082838e+07,50473.700000,2.205725e+07,2014.000000,2014.000000,2014.000000,1.284516e+07,389871.500000,...,2014.000000,2014.000000,2014.000000,2014.000000,9.838521e+07,1.394974e+06,8.722823e+07,2014.000000,2014.000000,2014.000000


In [30]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 45 columns):
BiomassC             265 non-null int64
CENSUSPOP            53 non-null float64
CoalC                265 non-null int64
CoalE                265 non-null float64
CoalP                265 non-null int64
CoalPrice            265 non-null float64
Coast                260 non-null float64
Division             260 non-null float64
ElecC                265 non-null int64
ElecE                265 non-null float64
ElecPrice            265 non-null float64
FossFuelC            265 non-null int64
GDP                  265 non-null float64
GDPQ1                265 non-null int64
GDPQ2                265 non-null int64
GDPQ3                265 non-null int64
GDPQ4                265 non-null int64
GDP_per_cap          265 non-null int64
GeoC                 265 non-null int64
GeoP                 265 non-null int64
Great Lakes          260 non-null float64
HydroC               265 non-null int6